<div style="display: flex; justify-content: space-between; align-items: center; padding: 8px 16px; background: #F8F9FA; border-bottom: 2px solid #E0E0E0; margin: 0; line-height: 1;">
    <div style="font-size: 14px; color: #666;">
        <span style="font-weight: bold; color: #333;">{SOURCE_PLATFORM} → Databricks Migration</span>
        <span style="margin-left: 8px; color: #999;">|</span>
        <span style="margin-left: 8px;">02 - Design</span>
    </div>
    <div style="display: flex; align-items: center; gap: 8px;">
        <img src="https://cdn.simpleicons.org/snowflake/29B5E8" width="24" height="24"/>
        <span style="color: #999; font-size: 16px;">→</span>
        <img src="https://cdn.simpleicons.org/databricks/FF3621" width="24" height="24"/>
    </div>
</div>


<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# Security and Access Design

## Overview

This module covers security architecture, access control patterns, identity management, and compliance frameworks for your Databricks lakehouse. Security design ensures data protection while enabling appropriate access for users and applications.

## Learning Objectives

By the end of this lesson, you will be able to:
- Implement Role-Based Access Control (RBAC) in Unity Catalog
- Configure Attribute-Based Access Control (ABAC) for dynamic policies
- Apply row-level and column-level security
- Design identity and authentication strategies
- Implement audit logging and compliance controls
- Map {SOURCE_PLATFORM} security models to Databricks

## Security Architecture Overview

Unity Catalog provides comprehensive security at multiple layers.

<br />
<div class="mermaid">
flowchart TB
    subgraph SECURITY["Security Layers"]
        direction TB
        A["Identity & Authentication<br/><i>SSO, MFA, Service Principals</i>"]
        B["Access Control (RBAC/ABAC)<br/><i>Catalog, Schema, Table privileges</i>"]
        C["Data Protection<br/><i>Row filters, column masks</i>"]
        D["Network Security<br/><i>Private Link, IP allowlists</i>"]
        E["Audit & Compliance<br/><i>System tables, log delivery</i>"]
    end
    A --> B --> C --> D --> E
    style SECURITY fill:#fff,stroke:#FF3621,stroke-width:2px
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

| Layer | Components | Protection |
|-------|------------|------------|
| **Identity** | SSO, MFA, Service Principals | Verify who is accessing |
| **Access Control** | RBAC, ABAC, grants | Control what they can access |
| **Data Protection** | Row/column security, encryption | Protect sensitive data |
| **Network** | Private Link, firewall | Secure connectivity |
| **Audit** | System tables, logs | Track and monitor access |

## Role-Based Access Control (RBAC)

RBAC grants permissions to users and groups on specific objects.

### Privilege Model

Unity Catalog privileges follow a hierarchical model:

| Level | Key Privileges | Grants Access To |
|-------|----------------|------------------|
| **Metastore** | `CREATE CATALOG`, `CREATE STORAGE CREDENTIAL`, `CREATE CONNECTION` | Account-level operations |
| **Catalog** | `USAGE`, `CREATE SCHEMA`, `USE CATALOG` | Catalog visibility and schema creation |
| **Schema** | `USAGE`, `CREATE TABLE`, `CREATE VIEW`, `CREATE FUNCTION` | Schema visibility and object creation |
| **Table/View** | `SELECT`, `MODIFY`, `READ FILES`, `WRITE FILES` | Data access and modification |
| **Volume** | `READ VOLUME`, `WRITE VOLUME` | File access |

### Grant Syntax

<div class="code-block" data-language="sql">
-- Grant catalog access
GRANT USAGE ON CATALOG prod TO `analysts`;

-- Grant schema access
GRANT USAGE ON SCHEMA prod.gold_analytics TO `analysts`;

-- Grant table select
GRANT SELECT ON TABLE prod.gold_analytics.daily_sales TO `analysts`;

-- Grant modify for data engineers
GRANT SELECT, MODIFY ON SCHEMA prod.silver_customers TO `data-engineers`;

-- Grant all privileges to owners
GRANT ALL PRIVILEGES ON SCHEMA prod.bronze_salesforce TO `salesforce-team`;
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

### Group-Based Access

Manage access through groups, not individual users.

<div class="code-block" data-language="sql">
-- Create logical groups
-- (Actually created in Account Console or via API)

-- Grant to groups, not individuals
GRANT USAGE ON CATALOG prod TO `all-users`;
GRANT SELECT ON SCHEMA prod.gold_analytics TO `analysts`;
GRANT SELECT, MODIFY ON SCHEMA prod.silver TO `data-engineers`;
GRANT ALL PRIVILEGES ON CATALOG dev TO `developers`;

-- Avoid granting to individuals
-- GRANT ... TO `john.doe@company.com`; -- ❌ Don't do this
</div>

| Group | Purpose | Typical Grants |
|-------|---------|----------------|
| **metastore-admins** | Platform administration | ALL PRIVILEGES on METASTORE |
| **catalog-admins-{catalog}** | Catalog management | ALL PRIVILEGES on CATALOG |
| **data-engineers** | ETL and pipeline development | SELECT, MODIFY on Bronze/Silver; CREATE TABLE |
| **data-analysts** | Analysis and reporting | SELECT on Silver/Gold; USAGE on SQL Warehouses |
| **data-scientists** | ML and advanced analytics | SELECT on Silver/Gold; CREATE MODEL |
| **bi-users** | Dashboard consumption | SELECT on Gold schemas only |

### Best Practices

| Practice | Rationale |
|----------|-----------|
| **Use groups** | Easier to manage, audit, and scale |
| **Principle of least privilege** | Grant only what's needed |
| **Separate by environment** | `prod-analysts`, `dev-analysts` |
| **Use `USAGE` + specific grants** | Explicit is better than implicit |
| **Document grant rationale** | Track why access was granted |

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Attribute-Based Access Control (ABAC)

ABAC uses attributes (user properties, data tags) for dynamic, policy-driven access control.

### User Attributes

Define custom attributes on users and groups:

<div class="code-block" data-language="sql">
-- REPLACEME: Set via Account Console or SCIM API
-- Example: User has attribute "department=sales"
</div>

### Row Filters (Row-Level Security)

Apply filters based on user attributes to restrict rows dynamically:

<div class="code-block" data-language="sql">
-- Create row filter function
CREATE FUNCTION prod.security.region_filter(region STRING)
RETURN IF(
  IS_MEMBER('regional-managers'),
  TRUE,  -- Regional managers see all
  region = current_user_attribute('region')  -- Others see only their region
);

-- Apply to table
ALTER TABLE prod.gold.sales
SET ROW FILTER prod.security.region_filter ON (region);
</div>

Now when users query the table, they automatically see only rows they're authorized for:

<div class="code-block" data-language="sql">
-- User with region=US-WEST sees only US-WEST rows
SELECT * FROM prod.gold.sales;
-- Automatically filtered!
</div>

### Column Masks (Dynamic Data Masking)

Mask sensitive columns based on user roles:

<div class="code-block" data-language="sql">
-- Create mask function
CREATE FUNCTION prod.security.email_mask(email STRING)
RETURN CASE
  WHEN IS_MEMBER('pii-readers') THEN email
  WHEN IS_MEMBER('analysts') THEN CONCAT(LEFT(email, 3), '***@***.com')
  ELSE NULL
END;

-- Apply to column
ALTER TABLE prod.silver.customers
ALTER COLUMN email SET MASK prod.security.email_mask;
</div>

| User Role | Query Result |
|-----------|--------------|
| `pii-readers` | `john.doe@company.com` |
| `analysts` | `joh***@***.com` |
| Others | `NULL` |

### Common Masking Patterns

| Pattern | Use Case | Implementation |
|---------|----------|----------------|
| **Full redaction** | Hide completely | Return NULL or '***' |
| **Partial masking** | Show prefix/suffix | `CONCAT(LEFT(col, 3), '***')` |
| **Hashing** | Consistent anonymization | `SHA2(col, 256)` |
| **Role-based** | Different views for roles | CASE with `IS_MEMBER()` |

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Identity and Authentication

Configure identity providers and authentication methods.

### Single Sign-On (SSO)

Integrate with corporate identity providers:

| Provider | Protocol | Configuration |
|----------|----------|---------------|
| **Azure AD** | SAML 2.0 / OIDC | Account Console → Settings → Authentication |
| **Okta** | SAML 2.0 | Configure Databricks app in Okta |
| **Google Workspace** | OIDC | OAuth client configuration |
| **Generic SAML** | SAML 2.0 | Custom IdP setup |

### User Provisioning (SCIM)

Automate user/group sync from IdP using SCIM 2.0:

| Task | Method |
|------|--------|
| **Enable SCIM** | Account Console → Settings → SCIM |
| **Generate Token** | Create account-level PAT for IdP |
| **Configure IdP** | Point to Databricks SCIM endpoint |
| **Sync Schedule** | Configure in IdP (e.g., hourly) |

> **Best Practice:** Use SCIM for all user management; avoid manual user creation.

### Service Principals

Machine identities for automation and CI/CD:

<div class="code-block" data-language="bash">
# Create service principal
databricks service-principals create \
  --display-name "etl-automation-sp"

# Generate OAuth token
databricks service-principals create-token \
  --service-principal-id <sp-id> \
  --lifetime-seconds 31536000 \
  --comment "ETL pipeline automation"
</div>

<div class="code-block" data-language="sql">
-- Grant privileges to service principal
GRANT USAGE ON CATALOG prod TO `etl-automation-sp`;
GRANT SELECT, MODIFY ON SCHEMA prod.bronze TO `etl-automation-sp`;
</div>

### Personal Access Tokens (PAT)

User-specific tokens for API access:

| Token Type | Scope | Use Case |
|------------|-------|----------|
| **Workspace Token** | Single workspace | Notebook automation, local development |
| **Account Token** | All workspaces | Cross-workspace operations, account admin tasks |

> **Security:** Set expiration dates; rotate regularly; never commit to source control.

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Network Security

Secure connectivity between users, workspaces, and data sources.

### Private Connectivity

| Cloud | Service | Use Case |
|-------|---------|----------|
| **AWS** | AWS PrivateLink | Private connection to workspace and metastore |
| **Azure** | Private Link | VNet injection for workspace isolation |
| **GCP** | Private Service Connect | Private connectivity to workspaces |

### IP Access Lists

Restrict workspace access by IP range:

<div class="code-block" data-language="bash">
# Add IP allowlist (Account Console → Settings → IP Access Lists)
# Or via API:
databricks ip-access-lists create \
  --label "Corporate Office" \
  --list-type ALLOW \
  --ip-addresses "203.0.113.0/24" "198.51.100.0/24"
</div>

### Firewall and Egress Control

Control outbound traffic from clusters:

| Direction | Control Method | Use Case |
|-----------|----------------|----------|
| **Ingress** | IP allowlists, Private Link | Restrict who can connect |
| **Egress** | VPC endpoints, NAT gateway | Control cluster outbound access |

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>


## Audit and Compliance

Monitor access and maintain compliance with regulations.

### Audit Logging

Unity Catalog captures all data access in system tables:

<div class="code-block" data-language="sql">
-- Query audit logs
SELECT 
  event_time,
  user_identity.email AS user,
  request_params.catalog AS catalog,
  request_params.schema AS schema,
  request_params.table AS table_name,
  audit_level,
  event_type
FROM system.access.audit
WHERE event_date >= CURRENT_DATE() - INTERVAL 7 DAYS
  AND event_type = 'getTable'
ORDER BY event_time DESC;
</div>

### System Tables

Query governance and usage data:

| System Table | Purpose | Example Query |
|--------------|---------|---------------|
| `system.access.audit` | All data access events | Who accessed what, when |
| `system.access.table_lineage` | Data lineage | Upstream/downstream dependencies |
| `system.billing.usage` | DBU consumption | Cost analysis by user/cluster |
| `system.query.history` | SQL query history | Performance analysis |

<div class="code-block" data-language="sql">
-- Identify users accessing PII
SELECT DISTINCT user_identity.email AS user
FROM system.access.audit
WHERE request_params.table = 'customers'
  AND request_params.full_name_arg LIKE '%ssn%';
</div>

### Log Delivery

Stream audit logs to your SIEM or data lake:

<div class="code-block" data-language="bash">
# Configure log delivery (Account Console)
# Delivers logs to S3, ADLS, or GCS
# JSON format, partitioned by date
</div>

### Compliance Frameworks

| Framework | Requirements | Databricks Support |
|-----------|--------------|-------------------|
| **SOC 2 Type II** | Security controls, audit trails | ✅ Certified |
| **HIPAA** | PHI protection, encryption, audit | ✅ BAA available |
| **GDPR** | Data residency, right to delete, consent | ✅ DPA, data deletion features |
| **PCI-DSS** | Cardholder data protection | ✅ With proper configuration |
| **FedRAMP** | Government security standards | ✅ FedRAMP Moderate (AWS) |

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Mapping {SOURCE_PLATFORM} Security to Databricks

Translate existing security models to Unity Catalog.

### Role Mapping

| {SOURCE_PLATFORM} Concept | Databricks Equivalent | Notes |
|---------------------------|----------------------|-------|
| **Role** | Group | Use account groups, not workspace-local |
| **User** | User (via SSO/SCIM) | Centrally managed at account level |
| **Service Account** | Service Principal | OAuth-based machine identity |
| **Role Hierarchy** | Group Nesting | Groups can contain other groups |

### Privilege Mapping

Map common {SOURCE_PLATFORM} privileges:

| {SOURCE_PLATFORM} Privilege | Unity Catalog Equivalent |
|----------------------------|-------------------------|
| `USAGE` on Database | `USAGE` on Catalog + Schema |
| `SELECT` on Table | `SELECT` on Table |
| `INSERT` on Table | `MODIFY` on Table |
| `UPDATE` on Table | `MODIFY` on Table |
| `DELETE` on Table | `MODIFY` on Table |
| `CREATE TABLE` | `CREATE TABLE` on Schema |
| `OWNERSHIP` | `OWNER` (transferable) |

### Security Feature Parity

| Feature | {SOURCE_PLATFORM} | Databricks | Migration Notes |
|---------|-------------------|------------|-----------------|
| **Row-Level Security** | REPLACEME | Row Filters | Translate policies to filter functions |
| **Column Masking** | REPLACEME | Column Masks | Recreate masking logic in functions |
| **Object Grants** | REPLACEME | Unity Catalog Grants | Direct translation |
| **RBAC** | REPLACEME | Groups + Grants | Map roles to groups |
| **Encryption at Rest** | REPLACEME | CMK Support | Configure in cloud provider |
| **Audit Logging** | REPLACEME | System Tables | Query system.access.audit |

## Security Design Patterns

Common patterns for enterprise security.

### Pattern 1: Environment Isolation

Separate dev/staging/prod with distinct security boundaries:

<div class="code-block" data-language="sql">
-- Separate catalogs per environment
CREATE CATALOG dev;
CREATE CATALOG staging;
CREATE CATALOG prod;

-- Grant broad access to dev
GRANT ALL PRIVILEGES ON CATALOG dev TO `developers`;

-- Restrict prod access
GRANT USAGE ON CATALOG prod TO `all-users`;
GRANT SELECT ON SCHEMA prod.gold_analytics TO `analysts`;
-- Only data engineers can modify prod
GRANT MODIFY ON SCHEMA prod.silver TO `data-engineers`;
</div>

### Pattern 2: Domain-Based Security

Organize by business domain with domain ownership:

<div class="code-block" data-language="sql">
-- Domain catalogs
CREATE CATALOG sales;
CREATE CATALOG marketing;
CREATE CATALOG finance;

-- Domain teams own their catalogs
ALTER CATALOG sales OWNER TO `sales-data-team`;
ALTER CATALOG marketing OWNER TO `marketing-data-team`;
ALTER CATALOG finance OWNER TO `finance-data-team`;

-- Cross-domain access via grants
GRANT SELECT ON SCHEMA sales.gold TO `marketing-analysts`;
</div>

### Pattern 3: Layered Access

Different access levels by medallion layer:

| Layer | Access Pattern | Granted To |
|-------|---------------|------------|
| **Bronze** | Read/write | Data engineers only |
| **Silver** | Read/write (engineers), read-only (analysts) | Engineers + analysts |
| **Gold** | Read-only | All users |

<div class="code-block" data-language="sql">
GRANT SELECT, MODIFY ON SCHEMA prod.bronze TO `data-engineers`;
GRANT SELECT, MODIFY ON SCHEMA prod.silver TO `data-engineers`;
GRANT SELECT ON SCHEMA prod.silver TO `analysts`;
GRANT SELECT ON SCHEMA prod.gold TO `all-users`;
</div>

### Pattern 4: Sensitive Data Protection

Apply row/column filters to PII and sensitive fields:

<div class="code-block" data-language="sql">
-- Column masking for PII
ALTER TABLE prod.silver.customers
ALTER COLUMN ssn SET MASK prod.security.ssn_mask;

ALTER TABLE prod.silver.customers
ALTER COLUMN email SET MASK prod.security.email_mask;

-- Row filtering for regional data
ALTER TABLE prod.silver.transactions
SET ROW FILTER prod.security.region_filter ON (region);
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Summary

### Security Design Checklist

| Area | Task | Status |
|------|------|--------|
| **Identity** | SSO configured | ☐ |
| | SCIM provisioning enabled | ☐ |
| | Service principals created | ☐ |
| **Access Control** | Groups defined and populated | ☐ |
| | RBAC grants applied | ☐ |
| | Privilege documentation created | ☐ |
| **Data Protection** | Row filters implemented for sensitive tables | ☐ |
| | Column masks applied to PII fields | ☐ |
| | Encryption at rest enabled (CMK) | ☐ |
| **Network** | Private Link configured (if required) | ☐ |
| | IP allowlists defined | ☐ |
| | Egress controls established | ☐ |
| **Audit** | System tables access configured | ☐ |
| | Log delivery to SIEM enabled | ☐ |
| | Compliance requirements documented | ☐ |

### Key Principles

| Principle | Implementation |
|-----------|----------------|
| **Least Privilege** | Grant only what's needed; use USAGE + specific grants |
| **Group-Based** | Manage access via groups, not individuals |
| **Defense in Depth** | Multiple security layers (identity, RBAC, data protection, network) |
| **Audit Everything** | Enable comprehensive logging; query system tables regularly |
| **Automate** | Use SCIM for provisioning; IaC for grant management |

### Next Steps

- Proceed to [**Module 03 - Execute**]($../03 - Execute/3.1 - Interoperability and Coexistence) to begin data migration

<div style="color: #FF3621; font-weight: bold; font-size: 2em; margin-bottom: 12px;">COURSE DEVELOPER (remove before publishing)</div>

### Template Customization

**Placeholders to replace:**
- `{SOURCE_PLATFORM}` - Source platform name
- `REPLACEME` - Platform-specific security features

**Platform-specific additions:**
- Map source platform roles/privileges to UC
- Document platform-specific security features
- Include migration scripts for security objects

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>
